# Introduction


- [Pandas](https://pandas.pydata.org/docs/) "an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language."
- [Hugging Face Transformers](https://huggingface.co/docs/transformers/en/index) "🤗 Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models."
- [Unsloth](https://github.com/unslothai/unsloth) "Finetune Llama 3, Mistral, Phi & Gemma LLMs 2-5x faster with 80% less memory"

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd

# Load the Data

In [3]:
data = pd.read_csv('data.csv', index_col = 0)
data

,Narratives,Synopsis
0,on the arrival we were busy on the tcas as the...,a be-40 flight crew experienced a tcas ra when...
1,we performed the pre-flight inspection of the ...,when the instructor and student pilots' aboard...
2,lft has an fdc notam which indicates as follow...,a corporate pilot reported that he was unable ...
3,flight crew briefed and understood the ruudy4 ...,a momentary erroneous right turn command as th...
4,after traveling internationally and requiring ...,a gulfstream flight crew felt compelled to des...
...,...,...
196,this training flight began mid afternoon local...,a c182rg nose landing gear collapsed during la...
197,nose gear failed to extend fully and lock down...,"a pc-12 nose landing gear failed to extend, so..."
198,"while at 4,500 ft cruise, heading north northw...",a c150 pilot declared an emergency and diverte...
199,"[i had a] normal balloon inflation, weather pe...",a ballon's rotator vent control lines became e...


# Try out the Models

In [5]:
conda create --name unsloth_env \
    python=3.10 \
    pytorch-cuda=<11.8/12.1> \
    pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers \
    -y
conda activate unsloth_env

pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes

SyntaxError: invalid syntax (2815489388.py, line 1)

In [4]:
from unsloth import FastLanguageModel

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit"
] # More models at https://huggingface.co/unsloth

MODEL_NAME = "unsloth/llama-3-8b-bnb-4bit" # REPLACE ME

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = 8192,
    load_in_4bit = True,
)
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

ModuleNotFoundError: No module named 'unsloth'

Querying base models

In [ ]:
def generate(input_text):
  inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, pad_token_id=tokenizer.eos_token_id)
  response = (tokenizer.batch_decode(outputs)[0])
  return response

In [ ]:
input_text = "Summarize Moby Dick"
generate(input_text)

'<|begin_of_text|>Summarize Moby Dick\n1. Moby Dick is the story of a man named Ishmael. Ishmael is a sailor who is obsessed with whales. Ishmael joins a whaling ship called the Pequod, which is captained by Captain Ahab. Captain Ahab is obsessed with killing a whale named Moby'

# Summarize a Narrative

In [ ]:
narratives = data.loc[0, 'Narratives']
narratives

'on the arrival we were busy on the tcas as there were multiple targets in our general vicinity when were given 6,000 msl. we were then given a descent to 5,000. upon landing i was told to call the tracon and was told i violated an altitude. after examination of the situation, we either answered a descend to 5,000 ft clearance intended for another airplane or replied to the clearance to descend to 6,000 ft and put 5,000 ft in the altitude window in error.     in either situation there was a conflict with other traffic and i understand it is not acceptable. i will request a copy of the tape for my own personal use and possible training to other people in the future. to prevent this from happening in the future i will make sure to look at the altitude window and point to verify the altitude assigned and not just repeat without visual confirmation. i am personally and professionally embarrassed by this mistake.,...while we were descending to 6,000 ft we were cleared to 5,000 ft and when w

In [ ]:
synopsis = data.loc[0, 'Synopsis']
synopsis

'a be-40 flight crew experienced a tcas ra when they descended to 5,000 when cleared to 6,000.'

In [ ]:
prompt = f"### Write a summary of the following Text: {narratives}"
prompt

'### Summarize the following Text: on the arrival we were busy on the tcas as there were multiple targets in our general vicinity when were given 6,000 msl. we were then given a descent to 5,000. upon landing i was told to call the tracon and was told i violated an altitude. after examination of the situation, we either answered a descend to 5,000 ft clearance intended for another airplane or replied to the clearance to descend to 6,000 ft and put 5,000 ft in the altitude window in error.     in either situation there was a conflict with other traffic and i understand it is not acceptable. i will request a copy of the tape for my own personal use and possible training to other people in the future. to prevent this from happening in the future i will make sure to look at the altitude window and point to verify the altitude assigned and not just repeat without visual confirmation. i am personally and professionally embarrassed by this mistake.,...while we were descending to 6,000 ft we w

In [ ]:
generate(prompt)

'<|begin_of_text|>### Summarize the following Text: on the arrival we were busy on the tcas as there were multiple targets in our general vicinity when were given 6,000 msl. we were then given a descent to 5,000. upon landing i was told to call the tracon and was told i violated an altitude. after examination of the situation, we either answered a descend to 5,000 ft clearance intended for another airplane or replied to the clearance to descend to 6,000 ft and put 5,000 ft in the altitude window in error.     in either situation there was a conflict with other traffic and i understand it is not acceptable. i will request a copy of the tape for my own personal use and possible training to other people in the future. to prevent this from happening in the future i will make sure to look at the altitude window and point to verify the altitude assigned and not just repeat without visual confirmation. i am personally and professionally embarrassed by this mistake.,...while we were descending

Prompt Engineering

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
The ASRS database is the world's largest repository of voluntary, confidential safety information provided by aviation's frontline personnel, including pilots, controllers, mechanics, flight attendants, and dispatchers. The database provides a foundation for specific products and subsequent research addressing a variety of aviation safety issues.
ASRS's database includes the narratives submitted by reporters (after they have been sanitized for identifying details). These narratives provide an exceptionally rich source of information for policy development, human factors research, education, training, and more. The database also contains coded information by expert analysts from the original report which is used for data retrieval and analyses.
Each report received by the ASRS is read by a minimum of two analysts that classify reports and diagnose the causes underlying each reported event. Their observations, and the original de-identified report, are then incorporated into the ASRS's database.
Write the ASRS Summary for the given Narratvives.

### Input:
{}

### Response:
{}"""
prompt = alpaca_prompt.format(narratives, '')
prompt

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nThe ASRS database is the world's largest repository of voluntary, confidential safety information provided by aviation's frontline personnel, including pilots, controllers, mechanics, flight attendants, and dispatchers. The database provides a foundation for specific products and subsequent research addressing a variety of aviation safety issues.\nASRS's database includes the narratives submitted by reporters (after they have been sanitized for identifying details). These narratives provide an exceptionally rich source of information for policy development, human factors research, education, training, and more. The database also contains coded information by expert analysts from the original report which is used for data retrieval and analyses.\nEach report received by the ASRS is read by a minimum of two ana

In [ ]:
generate(prompt)

"<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nThe ASRS database is the world's largest repository of voluntary, confidential safety information provided by aviation's frontline personnel, including pilots, controllers, mechanics, flight attendants, and dispatchers. The database provides a foundation for specific products and subsequent research addressing a variety of aviation safety issues.\nASRS's database includes the narratives submitted by reporters (after they have been sanitized for identifying details). These narratives provide an exceptionally rich source of information for policy development, human factors research, education, training, and more. The database also contains coded information by expert analysts from the original report which is used for data retrieval and analyses.\nEach report received by the ASRS is read by a m

# Fine-Tuning

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

Enable [PEFT](https://huggingface.co/docs/peft/en/index)

In [ ]:
peft_model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import Dataset, load_from_disk

dataset = load_from_disk('/content/drive/Shareddrives/2024.07.02 Fine-Tuning LLMs/dataset')['train']

trainer = SFTTrainer(
    model = peft_model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 15,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/160 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 160 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 15
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.841200
2,1.464600
3,1.697500
4,1.449400
5,1.477000
6,1.622100
7,1.652000
8,1.512400
9,1.522300
10,1.633400


In [ ]:
generate(prompt)

"<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nThe ASRS database is the world's largest repository of voluntary, confidential safety information provided by aviation's frontline personnel, including pilots, controllers, mechanics, flight attendants, and dispatchers. The database provides a foundation for specific products and subsequent research addressing a variety of aviation safety issues.\nASRS's database includes the narratives submitted by reporters (after they have been sanitized for identifying details). These narratives provide an exceptionally rich source of information for policy development, human factors research, education, training, and more. The database also contains coded information by expert analysts from the original report which is used for data retrieval and analyses.\nEach report received by the ASRS is read by a m